<a href="https://colab.research.google.com/github/satoshibasaki/competition/blob/main/signate_46th_beginner_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 【第46回_Beginner限定コンペ】携帯電話の機能データからの価格帯分類


携帯電話の機能から販売価格を分類しよう！

## ライブラリの読み込み

In [1]:
!pip install optuna optuna-integration catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.0 MB/s eta 0:00:00


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBClassifier
from sklearn.svm import SVC
# from lightgbm import LGBMClassifier
import lightgbm as lgb
import catboost as cb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
import optuna
import optuna.visualization as vis
# import optuna.integration.lightgbm as lgb
from optuna.integration import CatBoostPruningCallback

## データの読み込み

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dir = "/content/drive/MyDrive/Data/signate_46th_begginer"
df_train_all = pd.read_csv(dir + '/train.csv')
df_test = pd.read_csv(dir + '/test.csv')
df_test_id = df_test["id"]
sample_submission = pd.read_csv(dir + '/sample_submission.csv')

テストデータセットとサンプルファイルのデータ数が合わない。

id=1 がサンプルには無いのが原因。

## データの前処理

In [5]:
# id は不要なので落とす
df_train_all = df_train_all.drop('id', axis=1)
df_test = df_test.drop('id', axis=1)

# ターゲット変数 y は分けておく
y = df_train_all["price_range"]
X = df_train_all.drop("price_range", axis=1)  # ここから特徴量は X, df_test に対して作成・追加する

### 新しい特徴量の作成・追加

In [6]:
# スクリーン面積: sc_area
X["sc_area"] = np.multiply(X["sc_h"], X["sc_w"])
df_test["sc_area"] = np.multiply(df_test["sc_h"], df_test["sc_w"])

# 画面比率: sc_ratio => NG (inf)
# X["sc_ratio"] = np.divide(X["sc_h"], X["sc_w"])
# df_test["sc_ratio"] = np.divide(df_test["sc_h"], df_test["sc_w"])

# 連続通信時間比: talk_time_ratio
# X["talk_time_ratio"] = X["talk_time"] / X["battery_power"]
# df_test["talk_time_ratio"] = df_test["talk_time"] / df_test["battery_power"]

# ピクセル密度: px_density => 値が大きすぎてNG (inf)
# X["px_density"] = np.divide(np.multiply(X["px_height"], X["px_width"]), X["sc_area"])
# df_test["px_density"] = np.divide(np.multiply(df_test["px_height"], df_test["px_width"]), df_test["sc_area"])

# プロセッサ速度指数: ps_idx
X["ps_idx"] = np.multiply(X["clock_speed"], X["n_cores"])
df_test["ps_idx"] = np.multiply(df_test["clock_speed"], df_test["n_cores"])

# コア当たりのRAM: ram_per_core
# X["ram_per_core"] = np.divide(X["ram"], X["n_cores"])
# df_test["ram_per_core"] = np.divide(df_test["ram"], df_test["n_cores"])

# メモリ効率: mem_eff
# X["mem_eff"] = np.divide(X["ram"], X["mobile_wt"])
# df_test["mem_eff"] = np.divide(df_test["ram"], df_test["mobile_wt"])

# ストレージ効率: str_eff
# X["str_eff"] = np.divide(X["int_memory"], X["mobile_wt"])
# df_test["str_eff"] = np.divide(df_test["int_memory"], df_test["mobile_wt"])

# バッテリー密度: bp_wt
# X["bp_wt"] = np.divide(X["battery_power"], X["mobile_wt"])
# df_test["bp_wt"] = np.divide(df_test["battery_power"], df_test["mobile_wt"])

# カメラ品質指数: cq_idx
X["cq_idx"] = X["pc"] + X["fc"]
df_test["cq_idx"] = df_test["pc"] + df_test["fc"]

# 通信機能数: conn_ft_cnt
X["conn_ft_cnt"] = X["blue"] + X["four_g"] + X["three_g"] + X["wifi"]
df_test["conn_ft_cnt"] = df_test["blue"] + df_test["four_g"] + df_test["three_g"] + df_test["wifi"]

# 全体の機能数: all_ft_cnt
X["all_ft_cnt"] = X["blue"] + X["dual_sim"]+ X["touch_screen"] + X["four_g"] + X["three_g"] + X["wifi"]
df_test["all_ft_cnt"] = df_test["blue"]+ df_test["dual_sim"]+ df_test["touch_screen"] + df_test["four_g"] + df_test["three_g"] + df_test["wifi"]

In [7]:
# 特徴量同士の相関関係を表示
# corr = X.corr()
# plt.figure(figsize=(10, 8))  # サイズを大きく設定
# sns.heatmap(corr, annot=True, annot_kws={"size":6}, cmap='coolwarm')
# plt.show()

### 特徴量の尺度を揃える

In [8]:
std_columns = ['battery_power', 'clock_speed', 'fc',
        'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc',
        'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'sc_area',
        'ps_idx', 'cq_idx', 'conn_ft_cnt', 'all_ft_cnt',
        ]

# Standardize training data
sc = StandardScaler().fit(X[std_columns])
# X_train[std_columns] = sc.transform(X_train[std_columns])
X[std_columns] = sc.transform(X[std_columns])

# Standardize validation data
# X_valid[std_columns] = sc.transform(X_valid[std_columns])

# Standardize test data
df_test[std_columns] = sc.transform(df_test[std_columns])

### データセットを訓練・検証データに分割

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Optuna でモデルのハイパーパラメータをチューニングする

### 1. ランダムフォレスト

In [ ]:
def rf_objective(trial):

    # 調整したいハイパーパラメータの範囲
    max_depth = trial.suggest_int("max_depth", 2, 32)
    n_estimators = trial.suggest_int("n_estimators", 10, 200)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)

    # モデルの定義
    cls = RandomForestClassifier(
        max_depth=max_depth,
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

    score = cross_val_score(cls, X, y, cv=5, scoring='f1_macro')
    f1_score = score.mean()
    return score.mean()

study = optuna.create_study(direction='maximize')
study.optimize(rf_objective, n_trials=100)

print('Best trial: , ', study.best_trial.params)
print('Best score: , ', study.best_trial.value)

[I 2024-05-23 23:16:22,346] A new study created in memory with name: no-name-55e0b6bd-f92a-4aaa-86f8-a5b97f24296d
[I 2024-05-23 23:16:24,535] Trial 0 finished with value: 0.42198926499421174 and parameters: {'max_depth': 4, 'n_estimators': 111, 'min_samples_split': 7, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.42198926499421174.
[I 2024-05-23 23:16:25,773] Trial 1 finished with value: 0.45181148963561774 and parameters: {'max_depth': 12, 'n_estimators': 67, 'min_samples_split': 10, 'min_samples_leaf': 9}. Best is trial 1 with value: 0.45181148963561774.
[I 2024-05-23 23:16:28,697] Trial 2 finished with value: 0.4539271677665225 and parameters: {'max_depth': 26, 'n_estimators': 186, 'min_samples_split': 13, 'min_samples_leaf': 15}. Best is trial 2 with value: 0.4539271677665225.
[I 2024-05-23 23:16:30,148] Trial 3 finished with value: 0.4592756165246613 and parameters: {'max_depth': 26, 'n_estimators': 89, 'min_samples_split': 18, 'min_samples_leaf': 4}. Best is trial 3 with 

Best trial: ,  {'max_depth': 13, 'n_estimators': 166, 'min_samples_split': 18, 'min_samples_leaf': 13}
Best score: ,  0.47069390011351286


In [ ]:
# 探索履歴のプロット
opt_history = vis.plot_optimization_history(study)
opt_history.show()

# ハイパーパラメータの重要度のプロット
param_importances = vis.plot_param_importances(study)
param_importances.show()

[W 2024-05-24 00:52:24,695] You need to set up the pruning feature to utilize `plot_intermediate_values()`


In [10]:
# ベストパラメータを使ってモデルの訓練

# Best trial: ,  {'max_depth': 13, 'n_estimators': 166, 'min_samples_split': 18, 'min_samples_leaf': 13}
# Best score: ,  0.47069390011351286

params = {'max_depth': 13,
          'n_estimators': 166,
          'min_samples_split': 18,
          'min_samples_leaf': 13}

rf = RandomForestClassifier(**params, random_state=42, n_jobs=-1)
# rf_results = evaluate_model(rf, X, y)
rf.fit(X, y)

# テストデータでの推論
rf_pred = rf.predict(df_test)
# print(rf_pred.shape)
# print(rf_pred)

### 2. SVC

In [ ]:
def svc_objective(trial):

    # 調整したいハイパーパラメータの範囲
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf'])
    params = {
        'C': trial.suggest_float('C', 1e-6, 1e2, log=True),
        'kernel': kernel
    }

    # モデルの定義
    cls = SVC(**params, random_state=42)

    score = cross_val_score(cls, X, y, cv=5, scoring='f1_macro')
    f1_score = score.mean()
    return score.mean()

study = optuna.create_study(direction='maximize')
study.optimize(svc_objective, n_trials=100)

print('Best trial: ', study.best_trial.params)
print('Best score: ', study.best_trial.value)

[I 2024-05-24 01:05:48,305] A new study created in memory with name: no-name-f29a233b-7c8e-448d-ae60-1b403f16a758
[I 2024-05-24 01:05:48,750] Trial 0 finished with value: 0.35337904692495353 and parameters: {'kernel': 'rbf', 'C': 0.2306451718788334}. Best is trial 0 with value: 0.35337904692495353.
[I 2024-05-24 01:05:49,123] Trial 1 finished with value: 0.4007004261610317 and parameters: {'kernel': 'linear', 'C': 0.21202713758748176}. Best is trial 1 with value: 0.4007004261610317.
[I 2024-05-24 01:05:49,443] Trial 2 finished with value: 0.12962962962962962 and parameters: {'kernel': 'linear', 'C': 3.0236388132303163e-05}. Best is trial 1 with value: 0.4007004261610317.
[I 2024-05-24 01:05:49,917] Trial 3 finished with value: 0.12962962962962962 and parameters: {'kernel': 'rbf', 'C': 0.006477490868455745}. Best is trial 1 with value: 0.4007004261610317.
[I 2024-05-24 01:05:50,233] Trial 4 finished with value: 0.12962962962962962 and parameters: {'kernel': 'linear', 'C': 0.000133314733

Best trial: ,  {'kernel': 'rbf', 'C': 0.7253529746614299}
Best score: ,  0.4483449607237124


In [ ]:
# 探索履歴のプロット
opt_history = vis.plot_optimization_history(study)
opt_history.show()

# ハイパーパラメータの重要度のプロット
param_importances = vis.plot_param_importances(study)
param_importances.show()

In [15]:
# ベストパラメータを使ってモデルの訓練

# Best trial: ,  {'kernel': 'rbf', 'C': 0.7253529746614299}
# Best score: ,  0.4483449607237124

params = {'C': 0.7253529746614299,
          'kernel': 'rbf'}

svc = SVC(**params, probability=True, random_state=42)
# rf_results = evaluate_model(rf, X, y)
svc.fit(X, y)

# テストデータでの推論
svc_pred = svc.predict(df_test)

# print(svc_pred.shape)
# print(svc_pred)

### 3. XGBoost

In [ ]:
def xgb_objective(trial):

    # 調整したいハイパーパラメータの範囲
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 8),
        'max_depth': trial.suggest_int('max_depth', 1, 4),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
        'subsample': trial.suggest_float('subsample', 0.2, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.001, 0.1, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.001, 0.1, log=True),
        'gamma': trial.suggest_float('gamma', 0.0001, 0.1, log=True),
    }

    # モデルの定義
    cls = XGBClassifier(**params, objective='multiclass', random_state=42, n_jobs=-1)

    score = cross_val_score(cls, X, y, cv=5, scoring='f1_macro')
    f1_score = score.mean()
    return score.mean()

study = optuna.create_study(direction='maximize')
study.optimize(xgb_objective, n_trials=100)

print('Best trial: ', study.best_trial.params)
print('Best score: ', study.best_trial.value)

[I 2024-05-24 21:34:57,826] A new study created in memory with name: no-name-65668db2-cfc5-4091-b0eb-cbe564f91b3c
[I 2024-05-24 21:34:58,688] Trial 0 finished with value: 0.43504435127499475 and parameters: {'learning_rate': 0.01396808031100182, 'min_child_weight': 4, 'max_depth': 2, 'colsample_bytree': 0.9939550785703735, 'subsample': 0.5892207348340746, 'reg_alpha': 0.06304092413606736, 'reg_lambda': 0.0012005248082420073, 'gamma': 0.0009411814436477706}. Best is trial 0 with value: 0.43504435127499475.
[I 2024-05-24 21:34:59,727] Trial 1 finished with value: 0.4542157663258587 and parameters: {'learning_rate': 0.11533602460488761, 'min_child_weight': 3, 'max_depth': 3, 'colsample_bytree': 0.8754996475460597, 'subsample': 0.9110647299381514, 'reg_alpha': 0.0055692184650854544, 'reg_lambda': 0.017020711970240132, 'gamma': 0.04800226852175068}. Best is trial 1 with value: 0.4542157663258587.
[I 2024-05-24 21:35:01,031] Trial 2 finished with value: 0.459227983315205 and parameters: {'le

Best trial: ,  {'learning_rate': 0.12383207707591747, 'min_child_weight': 5, 'max_depth': 4, 'colsample_bytree': 0.46905238052158094, 'subsample': 0.8236466968757384, 'reg_alpha': 0.003685214348877713, 'reg_lambda': 0.05253824861610229, 'gamma': 0.0002250569171273546}
Best score: ,  0.4902677266384255


In [ ]:
# 探索履歴のプロット
opt_history = vis.plot_optimization_history(study)
opt_history.show()

# ハイパーパラメータの重要度のプロット
param_importances = vis.plot_param_importances(study)
param_importances.show()

In [11]:
# ベストパラメータを使ってモデルの訓練

# Best trial: {'learning_rate': 0.12383207707591747, 'min_child_weight': 5, 'max_depth': 4, 'colsample_bytree': 0.46905238052158094, 'subsample': 0.8236466968757384, 'reg_alpha': 0.003685214348877713, 'reg_lambda': 0.05253824861610229, 'gamma': 0.0002250569171273546}
# Best score: 0.491854649903965

params = {
    'learning_rate': 0.12383207707591747,
    'min_child_weight': 5,
    'max_depth': 4,
    'colsample_bytree': 0.46905238052158094,
    'subsample': 0.8236466968757384,
    'reg_alpha': 0.003685214348877713,
    'reg_lambda': 0.05253824861610229,
    'gamma': 0.0002250569171273546
    }

xgb_model = XGBClassifier(**params, objective='multiclass', random_state=42, n_jobs=-1)
xgb_model.fit(X, y)

# テストデータでの推論
xgb_pred = xgb_model.predict(df_test)

print(xgb_pred.shape)
print(xgb_pred)

(800,)
[3 0 3 1 0 3 0 2 1 1 2 2 3 1 2 3 1 3 0 1 2 1 2 3 1 1 3 1 3 2 0 3 3 2 1 2 0
 0 0 0 3 3 1 0 2 2 3 2 2 2 2 2 3 1 1 1 2 0 3 2 0 1 2 1 2 2 2 2 1 1 0 1 2 2
 3 1 0 0 2 2 1 1 0 2 2 3 2 1 0 0 3 2 1 3 1 2 1 0 2 2 0 1 0 2 2 0 3 2 3 1 2
 0 2 0 1 3 3 1 0 1 0 3 1 1 0 1 2 2 0 2 1 0 1 2 2 3 0 2 1 3 2 3 2 3 2 3 3 1
 2 1 2 2 2 2 3 1 2 2 3 3 2 3 2 1 2 3 0 3 1 1 2 3 1 3 2 1 2 1 2 1 2 2 0 0 2
 2 1 3 2 2 3 1 2 3 1 0 3 1 2 0 3 2 0 2 2 3 2 1 1 1 1 1 1 1 1 2 2 2 3 2 2 0
 1 1 1 1 1 2 1 1 3 1 2 0 0 3 2 2 2 2 2 0 0 0 3 0 0 2 3 0 0 3 2 1 3 2 0 1 3
 0 3 2 3 2 1 3 2 3 1 2 1 2 1 2 0 0 2 1 2 1 1 2 0 3 2 1 2 3 0 3 2 2 1 3 1 3
 2 3 2 2 0 1 2 0 1 2 2 1 3 3 2 1 1 2 2 1 2 2 0 1 2 1 2 3 0 2 1 3 1 2 0 1 1
 1 2 1 3 2 2 2 3 3 3 2 1 3 1 2 3 3 2 2 2 0 3 1 3 3 2 1 0 1 1 1 1 0 2 1 3 2
 1 3 1 2 2 1 3 3 3 1 1 2 3 1 2 2 1 2 3 2 1 3 2 0 3 2 3 2 3 2 0 0 3 0 1 3 3
 2 1 2 1 0 1 3 2 1 3 2 1 3 2 2 0 2 1 0 1 3 0 1 3 2 1 1 2 0 1 2 0 0 3 3 1 2
 3 3 1 1 3 1 2 0 2 2 3 3 0 3 2 1 2 3 2 2 2 3 0 1 2 0 1 2 0 2 2 3 3 2 1 0 2
 0 1 2 2 1 3 3 3 1

### 4. LightGBM

In [ ]:
import time
start = time.time()

def objective(trial):
    # X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    # dtrain = lgb.Dataset(X_train, label=y_train)
    # dvalid = lgb.Dataset(X_valid, label=y_valid, reference=dtrain)

    param = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'num_class': 4,
        'num_iterations': trial.suggest_int('num_iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1.0, log=True),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'random_state': 42,
        'verbose': -1,
        'n_jobs': -1
    }

    # verbose_eval = 0

    # gbm = lgb.train(param, dtrain, valid_sets=[dvalid],
    #                 callbacks=[
    #         lgb.early_stopping(stopping_rounds=10, verbose=True),
    #         lgb.log_evaluation(verbose_eval)
    #     ])
    gbm = LGBMClassifier(**param)
    gbm.fit(X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            callbacks=[
            lgb.early_stopping(stopping_rounds=20, verbose=True),
            lgb.log_evaluation(100),
        ])

    # preds = gbm.predict(X_valid).astype(np.float32)
    # print(preds[:5])
    # pred_labels = np.argmax(preds, axis=1)
    preds = gbm.predict(X_valid)
    # f1_macro = f1_score(y_valid, pred_labels, average='macro')
    f1_macro = f1_score(y_valid, preds, average='macro')
    return f1_macro

study = optuna.create_study(direction='maximize')
# optuna.logging.set_verbosity(optuna.logging.CRITICAL)
study.optimize(objective, n_trials=100)

print('\nNumber of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best score: ', study.best_trial.value)

print(f'所要時間{time.time() - start}秒')

[I 2024-05-27 21:32:20,706] A new study created in memory with name: no-name-af3f57a9-1e96-4383-b246-c77740a58f8b
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 20 rounds
[100]	valid_0's multi_logloss: 1.18212
[200]	valid_0's multi_logloss: 1.12175
[300]	valid_0's multi_logloss: 1.09505
[400]	valid_0's multi_logloss: 1.08061
[500]	valid_0's multi_logloss: 1.07168
[600]	valid_0's multi_logloss: 1.06538
[700]	valid_0's multi_logloss: 1.06077
[800]	valid_0's multi_logloss: 1.05684
[900]	valid_0's multi_logloss: 1.05397
Did not meet early stopping. Best iteration is:
[935]	valid_0's multi_logloss: 1.05279


[I 2024-05-27 21:32:26,423] Trial 0 finished with value: 0.48170476750478625 and parameters: {'num_iterations': 937, 'learning_rate': 0.009135873035331924, 'lambda_l1': 8.225592773803603, 'lambda_l2': 9.577449793796003, 'num_leaves': 17, 'feature_fraction': 0.41847116287475994, 'bagging_fraction': 0.9901757233349818, 'bagging_freq': 7, 'min_child_samples': 91}. Best is trial 0 with value: 0.48170476750478625.
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
[I 2024-05-27 21:32:26,601] Trial 1 finished with value: 0.4671054515961165 and parameters: {'num_iterations': 905, 'learning_rate': 0.18616576344661936, 'lambda_l1': 0.6639914211462616, 'lambda_l2': 9.892165531316575, 'num_leaves': 169, 'feature_fraction': 0.5777803892149629, 'bagging_fraction': 0.7764391986948767, 'bagging_freq': 6, 'min_child_samples': 24}. Bes

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[51]	valid_0's multi_logloss: 1.05689
Training until validation scores don't improve for 20 rounds


[I 2024-05-27 21:32:26,675] Trial 2 finished with value: 0.49157243746631774 and parameters: {'num_iterations': 234, 'learning_rate': 0.6517245519804242, 'lambda_l1': 7.930754253192161, 'lambda_l2': 5.567002125500021, 'num_leaves': 105, 'feature_fraction': 0.6644908976206259, 'bagging_fraction': 0.9319027170235565, 'bagging_freq': 2, 'min_child_samples': 24}. Best is trial 2 with value: 0.49157243746631774.
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

Early stopping, best iteration is:
[223]	valid_0's multi_logloss: 1.04295


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 20 rounds
[100]	valid_0's multi_logloss: 1.10293
[200]	valid_0's multi_logloss: 1.05758


[I 2024-05-27 21:33:32,300] Trial 99 finished with value: 0.5024528073129761 and parameters: {'num_iterations': 406, 'learning_rate': 0.015295547737502736, 'lambda_l1': 2.1338357315339547, 'lambda_l2': 1.4755532836618341, 'num_leaves': 47, 'feature_fraction': 0.7616219936437965, 'bagging_fraction': 0.6981449741874346, 'bagging_freq': 4, 'min_child_samples': 62}. Best is trial 97 with value: 0.537791922696224.


Early stopping, best iteration is:
[276]	valid_0's multi_logloss: 1.0453

Number of finished trials: 100
Best trial: {'num_iterations': 390, 'learning_rate': 0.02333208305281641, 'lambda_l1': 2.9738782603185556, 'lambda_l2': 1.5159321612358156, 'num_leaves': 14, 'feature_fraction': 0.7763349478313206, 'bagging_fraction': 0.6964851869230628, 'bagging_freq': 4, 'min_child_samples': 66}
Best score:  0.537791922696224
所要時間71.62061810493469秒


In [ ]:
# 探索履歴のプロット
opt_history = vis.plot_optimization_history(study)
opt_history.show()

# ハイパーパラメータの重要度のプロット
param_importances = vis.plot_param_importances(study)
param_importances.show()

In [13]:
# ベストパラメータを使ってモデルの訓練

# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# dtrain = lgb.Dataset(X_train, label=y_train)
# dvalid = lgb.Dataset(X_valid, label=y_valid, reference=dtrain)

# Best trial: {'num_iterations': 390, 'learning_rate': 0.02333208305281641, 'lambda_l1': 2.9738782603185556, 'lambda_l2': 1.5159321612358156, 'num_leaves': 14, 'feature_fraction': 0.7763349478313206, 'bagging_fraction': 0.6964851869230628, 'bagging_freq': 4, 'min_child_samples': 66}
# Best score:  0.537791922696224

param = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'num_class': 4,
    'num_iterations': 390,
    'learning_rate': 0.02333208305281641,
    'lambda_l1': 2.9738782603185556,
    'lambda_l2': 1.5159321612358156,
    'num_leaves': 14,
    'feature_fraction': 0.7763349478313206,
    'bagging_fraction': 0.6964851869230628,
    'bagging_freq': 4,
    'min_child_samples': 66,
    'random_state': 42,
    'verbose': -1
    }

# verbose_eval = 0
# lgb_model = lgb.train(param, dtrain, valid_sets=[dvalid],
#                 callbacks=[
#         lgb.early_stopping(stopping_rounds=10, verbose=True),
#         lgb.log_evaluation(verbose_eval)
#     ])

lgb_model = lgb.LGBMClassifier(**param)

# テストデータでの推論
# lgb_preds = lgb_model.predict(df_test).astype(np.float32)
# lgb_preds = np.argmax(lgb_preds, axis=1)

# print(lgb_preds.shape)
# print(lgb_preds)

### 5. CatBoost

In [ ]:
def objective(trial):

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    param = {
        'loss_function': 'MultiClass',
        # "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1.0, log=True),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
        "eval_metric": "Accuracy",
        "random_state": 42,
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    gbm = cb.CatBoostClassifier(**param)

    pruning_callback = CatBoostPruningCallback(trial, "Accuracy")
    gbm.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=0,
        early_stopping_rounds=20,
        callbacks=[pruning_callback],
    )

    # evoke pruning manually.
    pruning_callback.check_pruned()

    preds = gbm.predict(X_valid)
    # pred_labels = np.rint(preds)
    # accuracy = accuracy_score(y_valid, pred_labels)
    # accuracy = accuracy_score(y_valid, preds)
    f1_macro = f1_score(y_valid, preds, average='macro')

    # return accuracy
    return f1_macro

In [ ]:
study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize",
    )
study.optimize(objective, n_trials=100, timeout=600, n_jobs=-1)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-05-27 00:39:38,585] A new study created in memory with name: no-name-549e0cf4-56eb-4448-b442-01bad7eca2fe
<ipython-input-50-4975fbfa3d8d>:27: ExperimentalWarning:

CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.

<ipython-input-50-4975fbfa3d8d>:27: ExperimentalWarning:

CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.

[I 2024-05-27 00:39:38,958] Trial 1 finished with value: 0.12962962962962962 and parameters: {'learning_rate': 0.03215577985250393, 'colsample_bylevel': 0.014852591870153345, 'depth': 12, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.4114880133294928}. Best is trial 1 with value: 0.12962962962962962.
<ipython-input-50-4975fbfa3d8d>:27: ExperimentalWarning:

CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.

[I 2024-05-27 00:39:39,008] Trial 0 finished with value: 0.25952

Number of finished trials: 100
Best trial:
  Value: 0.5144574419815232
  Params: 
    learning_rate: 0.5740110283756216
    colsample_bylevel: 0.07233891150849797
    depth: 6
    boosting_type: Ordered
    bootstrap_type: Bayesian
    bagging_temperature: 1.9072492513603367


In [ ]:
# 探索履歴のプロット
opt_history = vis.plot_optimization_history(study)
opt_history.show()

# ハイパーパラメータの重要度のプロット
param_importances = vis.plot_param_importances(study)
param_importances.show()

In [14]:
# ベストパラメータを使ってモデルの訓練

# Best trial: {learning_rate: 0.5740110283756216
    # colsample_bylevel: 0.07233891150849797
    # depth: 6
    # boosting_type: Ordered
    # bootstrap_type: Bayesian
    # bagging_temperature: 1.9072492513603367}
# Best score:  0.5144574419815232

param = {
        'loss_function': 'MultiClass',
        'learning_rate': 0.5740110283756216,
        "colsample_bylevel": 0.07233891150849797,
        "depth": 6,
        "boosting_type": "Ordered",
        "bootstrap_type": "Bayesian",
        "used_ram_limit": "3gb",
        "eval_metric": "Accuracy",
        "random_state": 42
        }

cat_model = cb.CatBoostClassifier(**param)

# pruning_callback = CatBoostPruningCallback(trial, "Accuracy")
# cat_model.fit(
#     X_train,
#     y_train,
#     # X, y,
#     eval_set=[(X_valid, y_valid)],
#     verbose=2,
#     early_stopping_rounds=20,
#     callbacks=[pruning_callback],
# )

# テストデータでの推論
# cat_preds = np.reshape(cat_model.predict(df_test), -1)

# print(cat_preds.shape)
# print(cat_preds)

## StackingClassifier を使ってアンサンブル学習

In [24]:
estimators = [('rf', rf), ('xgb', xgb_model), ('lgb', lgb_model), ('cat', cat_model)]

stack = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    n_jobs=-1
)

stack.fit(X_train, y_train)

y_pred = stack.predict(X_valid)

# 評価メトリックの計算
accuracy = accuracy_score(y_valid, y_pred)
f1_macro = f1_score(y_valid, y_pred, average='macro')

# 結果の表示
print('Accuracy:', accuracy)
print('F1 Macro:', f1_macro)

Accuracy: 0.5333333333333333
F1 Macro: 0.4985685651342925


## モデルの評価： テストデータを用いた推論

In [25]:
stack.fit(X, y)  # 全データを用いて訓練

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(max_depth=13,
                                                       min_samples_leaf=13,
                                                       min_samples_split=18,
                                                       n_estimators=166,
                                                       n_jobs=-1,
                                                       random_state=42)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.46905238052158094,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enabl...
                                               feature_fraction=0.7763349478313206,
                                               lambda_l1=2.9738782603185556,
                                               lambda_l2=1.5159321612358156,
                                               learning_rate=0.02333208305281641,
                                               metric='multi_logloss',
                                               min_child_samples=66,
                                               num_class=4, num_iterations=390,
                                               num_leaves=14,
                                               objective='multiclass',
                                               random_state=42, verbose=-1)),
                               ('cat',
                                <catboost.core.CatBoostClassifier object at 0x7ba731a0b970>)],
                   final_estimator=LogisticRegression(), n_jobs=-1)

In [26]:
# テストデータを用いた推論

pred_test = stack.predict(df_test)

pred_test = pred_test.astype(int)  # float => int
print(pred_test[:20])

[3 0 3 1 1 3 0 1 1 1 2 2 3 1 2 1 1 3 3 1]


In [27]:
# submission ファイル作成

df_submission = pd.DataFrame({'id': df_test_id, 'price_range': pred_test})
df_submission.to_csv('submission.csv', index=False, header=None)